In [18]:
import csv
import avro.schema
from avro.datafile import DataFileWriter
from avro.io import DatumWriter
from collections import namedtuple


data = 'data/2015.q1-q3.by_industry/2015.q1-q3 10 Total, all industries.csv'

fields = ("area_fips",
         "own_code",
         "industry_code",
         "agglvl_code",
         "size_code",
         "year",
         "qtr",
         "disclosure_code",
         "area_title",
         "own_title",
         "industry_title",
         "agglvl_title",
         "size_title",
         "qtrly_estabs",
         "month1_emplvl",
         "month2_emplvl",
         "month3_emplvl",
         "total_qtrly_wages",
         "taxable_qtrly_wages",
         "qtrly_contributions",
         "avg_wkly_wage",
         "lq_disclosure_code",
         "lq_qtrly_estabs",
         "lq_month1_emplvl",
         "lq_month2_emplvl",
         "lq_month3_emplvl",
         "lq_total_qtrly_wages",
         "lq_taxable_qtrly_wages",
         "lq_qtrly_contributions",
         "lq_avg_wkly_wage",
         "oty_disclosure_code",
         "oty_qtrly_estabs_chg",
         "oty_qtrly_estabs_pct_chg",
         "oty_month1_emplvl_chg",
         "oty_month1_emplvl_pct_chg",
         "oty_month2_emplvl_chg",
         "oty_month2_emplvl_pct_chg",
         "oty_month3_emplvl_chg",
         "oty_month3_emplvl_pct_chg",
         "oty_total_qtrly_wages_chg",
         "oty_total_qtrly_wages_pct_chg",
         "oty_taxable_qtrly_wages_chg",
         "oty_taxable_qtrly_wages_pct_chg",
         "oty_qtrly_contributions_chg",
         "oty_qtrly_contributions_pct_chg",
         "oty_avg_wkly_wage_chg",
         "oty_avg_wkly_wage_pct_chg")


DataRecord = namedtuple('DataRecord', fields)


def read_data(path):
    with open(path, 'rU') as data:
        reader = csv.reader(data)
        for row in map(DataRecord._make, reader):
            yield row
            
def parse_schema(path="data/qcew_schema.avsc"):
    with open(path, 'r') as data:
        return avro.schema.parse(data.read())
    
def serialize_records(records, outpath = "qcew_data.avro"):
    schema = parse_schema()
    with open(outpath, 'wb') as out:
        writer = DataFileWriter(out, DatumWriter(), schema)
        for record in records:
            record = dict((f, getattr(record, f)) for f in record._fields)
            writer.append(record)

    
if __name__ == "__main__":
    reader = read_data(data)
    i = 0
    for row in reader:
        if i < 16:
            print row
            i += 1
        else:
            break
    serialize_records(reader)
    #print "loading in data with %i rows" % (len(reader))


DataRecord(area_fips='area_fips', own_code='own_code', industry_code='industry_code', agglvl_code='agglvl_code', size_code='size_code', year='year', qtr='qtr', disclosure_code='disclosure_code', area_title='area_title', own_title='own_title', industry_title='industry_title', agglvl_title='agglvl_title', size_title='size_title', qtrly_estabs='qtrly_estabs_count', month1_emplvl='month1_emplvl', month2_emplvl='month2_emplvl', month3_emplvl='month3_emplvl', total_qtrly_wages='total_qtrly_wages', taxable_qtrly_wages='taxable_qtrly_wages', qtrly_contributions='qtrly_contributions', avg_wkly_wage='avg_wkly_wage', lq_disclosure_code='lq_disclosure_code', lq_qtrly_estabs='lq_qtrly_estabs_count', lq_month1_emplvl='lq_month1_emplvl', lq_month2_emplvl='lq_month2_emplvl', lq_month3_emplvl='lq_month3_emplvl', lq_total_qtrly_wages='lq_total_qtrly_wages', lq_taxable_qtrly_wages='lq_taxable_qtrly_wages', lq_qtrly_contributions='lq_qtrly_contributions', lq_avg_wkly_wage='lq_avg_wkly_wage', oty_disclosur

AvroTypeException: The datum {'lq_total_qtrly_wages': '1.29', 'size_code': '0', 'lq_qtrly_contributions': '0.86', 'month2_emplvl': '362306', 'industry_title': 'Total, all industries', 'month3_emplvl': '361004', 'lq_month3_emplvl': '1.25', 'total_qtrly_wages': '4219253653', 'year': '2015', 'qtr': '2', 'oty_qtrly_contributions_pct_chg': '-10.8', 'oty_month2_emplvl_pct_chg': '-0.4', 'agglvl_code': '96', 'oty_qtrly_estabs_chg': '-50', 'industry_code': '10', 'size_title': 'All establishment sizes', 'oty_avg_wkly_wage_chg': '12', 'oty_month1_emplvl_chg': '-1581', 'oty_taxable_qtrly_wages_pct_chg': '-3.4', 'area_title': 'Alabama -- Statewide', 'own_code': '8', 'oty_month3_emplvl_chg': '-1162', 'lq_avg_wkly_wage': '1.04', 'oty_avg_wkly_wage_pct_chg': '1.4', 'oty_month1_emplvl_pct_chg': '-0.4', 'oty_qtrly_contributions_chg': '-37684', 'oty_month3_emplvl_pct_chg': '-0.3', 'oty_taxable_qtrly_wages_chg': '-733689', 'own_title': 'Total Government', 'oty_month2_emplvl_chg': '-1475', 'lq_month2_emplv